In [1]:
from torch.utils.data import DataLoader, Subset
train_data = DataLoader("/kaggle/input/inaturalist/inaturalist_12K/train")
val_data = DataLoader("/kaggle/input/inaturalist/inaturalist_12K/val")

In [2]:
!pip install torch


In [3]:
import numpy as np
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms #To manage dataset
from torch.utils.data import DataLoader, Subset #To load and transform data
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger # As pytorch.lightning allows direct logging into wandb
#from sklearn.model_selection import StratifiedShuffleSplit '''Check with TAs'''

class CNN(pl.LightningModule):
    def __init__(self,config):
        super().__init__() #using LightningModule to assign hyperparams
        self.save_hyperparameters()

        self.conv_blk == nn.ModuleList()
        input_ch = 3 #RGB images
        current_filters = config.filter_base #Filter Number & Strategies from sweep config
        for i in range(5): #Create five consecutive convolution blocks with configurable strategies
            self.conv_blk.append(
                nn.Conv2d(input_ch, current_filters, kernel_size = config.filter_size, padding = filter.size//2) #Use filter size from sweep config, padding = floor(0.5*filter)
            )
            if config.batch_norm==True:
                self.conv_blk.append(nn.BatchNorm2d(current_filters))
            #Define organization logic for filters in subsequent layers from sweep config metric filter_org
            if config.filter_org == "double":
                input_ch = current_filters
                current_filter *= 2 #Double number of filters
            elif config.filter_org == "halve":
                input_ch = current_filters
                current_filters = max(8, current_filters//2) #Ensuring a minimum of 8 filters
            else: #No filter strategies
                input_ch = current_filters
        #Calculate what the dimensions for the dense layer inputs would turn out to be
        with torch.no_grad():
            dummy_layer = torch.randn(1,3,224,224) #mimic input
            for layer in self.conv_blk:
                if isinstance(layer,nn.Conv2d): #Whenever convolution layer 
                    dummy_layer = F.max_pool2d(F.relu(layer(dummy_layer)),2) #MaxPool activation function layer after convolutions
                    self.dense_input = dummy_layer.view(1,-1).size(1) #Reshape to find size
        #Dense Connections. Linear->Dropout->Activation->Output
        self.dense = nn.Sequential(
            nn.Linear(self.dense_input,config.dense_neurons),
            nn.Dropout(config.dropout),
            self.actv(config.dense_actv),
            nn.Linear(config.dense_neurons, 10)
        )
        def actv(self,name): #Choosing activation func
            actv={
                "relu": nn.ReLU(),
                "gelu": nn.GELU(),
                "silu": nn.SiLU(),
                "mish": nn.Mish()
            }
            return actv[name.lower()]

        def forward(self,x): #Explicit Lightning Module Method Forward
            for layer in self.conv_blocks:
                if isinstance(layer, nn.Conv2d):
                    x = F.max_pool2d(self.actv(self.hparams.config.conv_actv)(layer(x)), 2)
                else:
                    x = layer(x)
                x = x.view(x.size(0),-1) #Flatten
                return self.dense(x) #Invoke Dense layers

        def training_step(self,batch,batch_idx):
            x, y = batch
            y_hat = self(x)
            loss = F.cross_entropy(y_hat,y)
            self.log("train_loss",loss)
            return loss

        def validation_step(self, batch, batch_idx):
            x, y = batch
            y_hat = self(x)
            loss = F.cross_entropy(y_hat,y)
            acc = (y_hat.argmax(dim=1) == y).float().mean() #Mean fractional accuracy
            self.log("val_loss",loss)
            self.log("val_acc", acc)
            return {"val_loss":loss, "val_acc":acc}

        def configure_optimizers(self): #Another pl method. We use Adam optim
            return torch.optim.Adam(self.parameters(), lr = self.hparams.config.lr)
                    


